In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load dataset
dataset = pd.read_csv('Thane_lstm_10.csv')
data = dataset.values
data = data.astype('float32')

In [3]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

In [4]:
# Split data into training and testing sets
train_size = int(len(data) * 0.67)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]

In [5]:
# Create function to create LSTM model
def create_model():
    model = Sequential()
    model.add(LSTM(50, input_shape=(1, 1)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [6]:
# Reshape input data to be 3-dimensional
X_train = np.reshape(train, (train.shape[0], 1, train.shape[1]))
X_test = np.reshape(test, (test.shape[0], 1, test.shape[1]))

In [7]:
# Create and fit the LSTM model
model = create_model()
model.fit(X_train, train, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
1027/1027 - 20s - loss: 0.0193 - 20s/epoch - 20ms/step
Epoch 2/100
1027/1027 - 6s - loss: 0.0010 - 6s/epoch - 6ms/step
Epoch 3/100
1027/1027 - 7s - loss: 0.0010 - 7s/epoch - 6ms/step
Epoch 4/100
1027/1027 - 7s - loss: 7.5296e-04 - 7s/epoch - 6ms/step
Epoch 5/100
1027/1027 - 7s - loss: 8.0604e-04 - 7s/epoch - 6ms/step
Epoch 6/100
1027/1027 - 7s - loss: 6.1687e-04 - 7s/epoch - 6ms/step
Epoch 7/100
1027/1027 - 7s - loss: 5.9683e-04 - 7s/epoch - 6ms/step
Epoch 8/100
1027/1027 - 7s - loss: 5.2795e-04 - 7s/epoch - 6ms/step
Epoch 9/100
1027/1027 - 6s - loss: 4.3831e-04 - 6s/epoch - 6ms/step
Epoch 10/100
1027/1027 - 7s - loss: 4.6078e-04 - 7s/epoch - 6ms/step
Epoch 11/100
1027/1027 - 6s - loss: 4.1173e-04 - 6s/epoch - 6ms/step
Epoch 12/100
1027/1027 - 6s - loss: 3.8691e-04 - 6s/epoch - 6ms/step
Epoch 13/100
1027/1027 - 6s - loss: 4.3749e-04 - 6s/epoch - 6ms/step
Epoch 14/100
1027/1027 - 6s - loss: 3.2112e-04 - 6s/epoch - 6ms/step
Epoch 15/100
1027/1027 - 6s - loss: 3.3494e-04 - 6s/

In [8]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

16/16 [==============================] - 0s 4ms/step


In [9]:
train_predict = scaler.inverse_transform(train_predict)
train = scaler.inverse_transform(train)
test_predict = scaler.inverse_transform(test_predict)
test = scaler.inverse_transform(test)

In [10]:
"""train_rmse = np.sqrt(np.mean(np.power((train - train_predict), 2)))
test_rmse = np.sqrt(np.mean(np.power((test - test_predict), 2)))
print('Train RMSE: %.2f' % train_rmse)
print('Test RMSE: %.2f' % test_rmse)"""

from sklearn.metrics import r2_score
import math
from sklearn.metrics import mean_squared_error
train_rmse = np.sqrt(mean_squared_error(train, train_predict))
test_rmse = np.sqrt(mean_squared_error(test, test_predict))
print('R2 Score_train:', r2_score(train, train_predict))
print('R2 Score_test:', r2_score(test, test_predict))

print('Train RMSE: %.2f' % train_rmse)
print('Test RMSE: %.2f' % test_rmse)

R2 Score_train: 0.9991052473883744
R2 Score_test: 0.9984622801656868
Train RMSE: 2.18
Test RMSE: 2.95


In [11]:
df=pd.DataFrame({'Actual':train.tolist(),'Predicted from lstm':train_predict.tolist()})
df.to_excel('lstm_10.xlsx')
df

,Actual,Predicted from lstm
0,[253.9600067138672],[249.1233673095703]
1,[245.71998596191406],[241.2428436279297]
2,[272.5],[266.74664306640625]
3,[335.1199951171875],[324.94873046875]
4,[245.47999572753906],[241.01290893554688]
...,...,...
1022,[230.85000610351562],[226.95433044433594]
1023,[244.4499969482422],[240.0258026123047]
1024,[229.55999755859375],[225.71095275878906]
1025,[295.0299987792969],[287.9393005371094]


In [12]:
df=pd.DataFrame({'Actual':test.tolist(),'Predicted from lstm':test_predict.tolist()})
df.to_excel('lstm_10_test.xlsx')
df

,Actual,Predicted from lstm
0,[260.3299865722656],[255.19577026367188]
1,[256.04998779296875],[251.11766052246094]
2,[187.19000244140625],[184.59237670898438]
3,[112.8399887084961],[111.70287322998047]
4,[149.52999877929688],[147.72544860839844]
...,...,...
501,[100.42000579833984],[99.51643371582031]
502,[120.79590606689453],[119.51394653320312]
503,[120.79590606689453],[119.51394653320312]
504,[120.79590606689453],[119.51394653320312]
